# Modelling for ML-Olympiad
Notebook de modelagem para predição de notas do ENEM.

In [1]:
import os.path
import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
# from google.colab import drive

In [2]:
# drive.mount('/content/drive')
# data_source = '/content/drive/MyDrive/Projetos/ML_Olympiad'
data_source = './data'
train_file_path = os.path.join(data_source, 'train.csv')

In [3]:
df = pd.read_csv(train_file_path)
df.shape

(3311925, 118)

In [4]:
categorical_columns = [
    # Dados do Participante
    'SG_UF_RESIDENCIA',
    'TP_SEXO',
    'TP_ESTADO_CIVIL',
    'TP_COR_RACA',
    'TP_NACIONALIDADE',
    'SG_UF_NASCIMENTO',
    'TP_ST_CONCLUSAO',
    'TP_ANO_CONCLUIU',
    'TP_ESCOLA',
    'TP_ENSINO',
    'IN_TREINEIRO',
    # Dados da escola
    'SG_UF_ESC',
    'TP_DEPENDENCIA_ADM_ESC',
    'TP_LOCALIZACAO_ESC',
    'TP_SIT_FUNC_ESC',
    # Dados do pedido de atendimento especializado
    'IN_BAIXA_VISAO',
    'IN_CEGUEIRA',
    'IN_SURDEZ',
    'IN_DEFICIENCIA_AUDITIVA',
    'IN_SURDO_CEGUEIRA',
    'IN_DEFICIENCIA_FISICA',
    'IN_DEFICIENCIA_MENTAL',
    'IN_DEFICIT_ATENCAO',
    'IN_DISLEXIA',
    'IN_DISCALCULIA',
    'IN_AUTISMO',
    'IN_VISAO_MONOCULAR',
    'IN_OUTRA_DEF',
    # Dados do pedido de atendimento específico
    'IN_GESTANTE',
    'IN_LACTANTE',
    'IN_IDOSO',
    'IN_ESTUDA_CLASSE_HOSPITALAR',
    # Dados dos pedidos de recursos especializados e específicos para realização das provas
    'IN_SEM_RECURSO',
    'IN_BRAILLE',
    'IN_AMPLIADA_24',
    'IN_AMPLIADA_18',
    'IN_LEDOR',
    'IN_ACESSO',
    'IN_TRANSCRICAO',
    'IN_LIBRAS',
    'IN_TEMPO_ADICIONAL',
    'IN_LEITURA_LABIAL',
    'IN_MESA_CADEIRA_RODAS',
    'IN_MESA_CADEIRA_SEPARADA',
    'IN_APOIO_PERNA',
    'IN_GUIA_INTERPRETE',
    'IN_COMPUTADOR',
    'IN_CADEIRA_ESPECIAL',
    'IN_CADEIRA_CANHOTO',
    'IN_CADEIRA_ACOLCHOADA',
    'IN_PROVA_DEITADO',
    'IN_MOBILIARIO_OBESO',
    'IN_LAMINA_OVERLAY',
    'IN_PROTETOR_AURICULAR',
    'IN_MEDIDOR_GLICOSE',
    'IN_MAQUINA_BRAILE',
    'IN_SOROBAN',
    'IN_MARCA_PASSO',
    'IN_SONDA',
    'IN_MEDICAMENTOS',
    'IN_SALA_INDIVIDUAL',
    'IN_SALA_ESPECIAL',
    'IN_SALA_ACOMPANHANTE',
    'IN_MOBILIARIO_ESPECIFICO',
    'IN_MATERIAL_ESPECIFICO',
    'IN_NOME_SOCIAL',
    # Dados do local de aplicação da prova
    'SG_UF_PROVA',
    # Dados da prova objetiva
    'TP_PRESENCA_CN',
    'TP_PRESENCA_CH',
    'TP_PRESENCA_LC',
    'TP_PRESENCA_MT',
    'TP_LINGUA',
    # Dados da redação
    'TP_STATUS_REDACAO',
    # Dados do questionário socioeconômico
    'Q001',
    'Q002',
    'Q003',
    'Q004',
    'Q006',
    'Q007',
    'Q008',
    'Q009',
    'Q010',
    'Q011',
    'Q012',
    'Q013',
    'Q014',
    'Q015',
    'Q016',
    'Q017',
    'Q018',
    'Q019',
    'Q020',
    'Q021',
    'Q022',
    'Q023',
    'Q024',
    'Q025'
]

In [5]:
numerical_columns = [
    'NU_IDADE',
    'Q005'
]

target_columns = [
    'NU_NOTA_CN',
    'NU_NOTA_CH',
    'NU_NOTA_LC',
    'NU_NOTA_MT',
    'NU_NOTA_REDACAO',
]

In [6]:
df[categorical_columns] = df[categorical_columns].astype('category')

In [7]:
df.columns[df.isna().sum() > 0]

Index(['NU_IDADE', 'CO_MUNICIPIO_NASCIMENTO', 'NO_MUNICIPIO_NASCIMENTO',
       'CO_UF_NASCIMENTO', 'SG_UF_NASCIMENTO', 'TP_ENSINO', 'CO_ESCOLA',
       'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC',
       'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC',
       'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
       'TP_STATUS_REDACAO', 'NU_NOTA_REDACAO'],
      dtype='object')

In [8]:
df[df.columns[df.isna().sum() > 0]].isna().sum() / df.shape[0]

NU_IDADE                   0.000010
CO_MUNICIPIO_NASCIMENTO    0.028552
NO_MUNICIPIO_NASCIMENTO    0.028552
CO_UF_NASCIMENTO           0.028552
SG_UF_NASCIMENTO           0.028552
TP_ENSINO                  0.435267
CO_ESCOLA                  0.774941
CO_MUNICIPIO_ESC           0.774941
NO_MUNICIPIO_ESC           0.774941
CO_UF_ESC                  0.774941
SG_UF_ESC                  0.774941
TP_DEPENDENCIA_ADM_ESC     0.774941
TP_LOCALIZACAO_ESC         0.774941
TP_SIT_FUNC_ESC            0.774941
NU_NOTA_CN                 0.271985
NU_NOTA_CH                 0.230175
NU_NOTA_LC                 0.230175
NU_NOTA_MT                 0.271985
TP_STATUS_REDACAO          0.230176
NU_NOTA_REDACAO            0.230176
dtype: float64

# Construção da Pipeline

In [9]:
from math import sqrt
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [10]:
numeric_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

categorical_transformer = OneHotEncoder(drop='if_binary', handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
      ('num', numeric_transformer, numerical_columns),
      ('cat', categorical_transformer, categorical_columns)
    ]
)

In [11]:
X, y = df[numerical_columns + categorical_columns], df[target_columns]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=24)

# Baseline

In [12]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [13]:
from sklearn.linear_model import Lasso

rgs = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('regressor', Lasso(random_state=24))
    ]
)
rgs.fit(X_train, y_train.fillna(0))
y_pred = rgs.predict(X_test)
print('MAE:', mean_absolute_error(y_test.fillna(0), y_pred))
print('RMSE:', sqrt(mean_squared_error(y_test.fillna(0), y_pred)))
print('R2:', r2_score(y_test.fillna(0), y_pred))

In [13]:
from sklearn.linear_model import Lasso

rgs = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('regressor', Lasso(random_state=24))
    ]
)

kfold = KFold(n_splits=5, shuffle=True, random_state=24)
scores = []
for train_idx, test_idx in tqdm.tqdm(kfold.split(X_train, y_train)):
    rgs.fit(X_train.loc[train_idx], y_train.loc[train_idx].fillna(0))
    y_pred = rgs.predict(X_train.loc[test_idx])
    scores.append({
        'MAE': mean_absolute_error(y_train.loc[test_idx], y_pred),
        'RMSE': sqrt(mean_squared_error(y_train.loc[test_idx], y_pred)),
        'R2': r2_score(y_train.loc[test_idx], y_pred),
    })

pd.DataFrame(data=scores).describe()

0it [00:00, ?it/s]